# Econometrics II: Homework 5
## Eric Schulman
## November 15, 2018

In [1]:
import pandas as pd
import numpy as np
import math

import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS

### Part 1 - Part 4

Setting up the hypothetical data

In [2]:
sigma = .1
var = [[1, sigma ], [sigma , 1]]
mean = [0., 0.]
N = int(1e6)

ey, ex = np.random.multivariate_normal(mean, var, size = N).transpose()

In [3]:
pz = .3

z = np.random.binomial(1,pz,N).astype(np.float)

In [4]:
beta_0x = -.5
beta_1x = 2

x = (beta_0x + beta_1x*z + ex > 0).astype(np.float) 

In [5]:
beta_0y = 1
beta_1y = 1

y = beta_0y + beta_1y*x + ey

### Part 5

$$E(y(1) -y(0)) = $$
$$E( (\beta_{0y} + \beta_{1jy} + \epsilon_{yj}) - (\beta_{0y} + \epsilon_{yj}) ) =$$
$$E(\beta_{1jy}) = \beta_{1y} = 1$$

### Part 6

* What happens to the OLS estimator applied to this data?

OLS is biased upward.

* Is that expected or unexpected?

This is expected since treatment is endogenously determined.

In [6]:
model = sm.OLS(y, sm.add_constant(x))
model = model.fit()
print model.summary()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.236
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                 3.097e+05
Date:                Thu, 15 Nov 2018   Prob (F-statistic):               0.00
Time:                        09:41:51   Log-Likelihood:            -1.4179e+06
No. Observations:             1000000   AIC:                         2.836e+06
Df Residuals:                  999998   BIC:                         2.836e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9440      0.001    671.325      0.0

### Part 7

Using the IV estimator recovers the average treatment effect.

In [7]:
model =  IV2SLS(y, sm.add_constant(x), sm.add_constant(z))
model = model.fit()
print model.summary()

                          IV2SLS Regression Results                           
Dep. Variable:                      y   R-squared:                       0.234
Model:                         IV2SLS   Adj. R-squared:                  0.234
Method:                     Two Stage   F-statistic:                 8.163e+04
                        Least Squares   Prob (F-statistic):               0.00
Date:                Thu, 15 Nov 2018                                         
Time:                        09:41:52                                         
No. Observations:             1000000                                         
Df Residuals:                  999998                                         
Df Model:                           1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0003      0.002    500.514      0.0

### Part 8 


* They are **compliers**:
    
    If $x_i = 1, z_i = 1$ and $x_i = 0, z_i = 0$ 
    
    Thus, $\epsilon_{xj} \geq -\beta_{0x} - \beta_{1x}$ and $\epsilon_{xj} < -\beta_{0x} $

*  They are **defiers**:
    
    If $x_i = 0, z_i = 1$ and $x_i = 1, z_i = 0$  
    
    Thus,  $\epsilon_{xj} < -\beta_{0x} - 
\beta_{1x}$ and $\epsilon_{xj} \geq - \beta_{0x}$

* They are never **takers**:
    
    If $x_i = 0, z_i = 0$ and $x_i = 0, z_i = 1$ 
    
    Thus, $\epsilon_{xj} < -\beta_{0x} $ and $\epsilon_{xj} < -\beta_{0x} -\beta_{1x} $

* They are **always takers**:
    
    If $x_i = 1, z_i = 0$ and $x_i = 1, z_i = 0$
    
    Thus, $\epsilon_{xj} \geq -\beta_{0x}$ and  $\epsilon_{xj} \geq -\beta_{0x} - \beta_{1x}$ 


### Part 9 

Above we can see that defiers have $\epsilon_{xj} < -\beta_{0x} - 
\beta_{1x}$ and $\epsilon_{xj} \geq - \beta_{0x}$


When   $\beta_{0x} < \beta_{0x} + \beta_{1x}$ we have no defiers. This is because the set of defiers is empty i.e. defiers have  $\beta_{0x} \geq - \epsilon_{xj}  > \beta_{0x} + \beta_{1x}$. If    $\beta_{0x} < \beta_{0x} + \beta_{1x}$, then this set is empty.

This is essentially a monotone treatment selection assumption. Getting the insturment makes you more likely to get treated. Depending upon the application, this is a reasonable assumption

### Part 10

In [8]:
#dummy variable for compliers
c = ( (- beta_0x > ex) * (ex > - beta_0x -  beta_1x) ).astype(np.float) 
print 'compliers: %s'% (c.sum()/N)

#dummy variable for defiers
d = ( (- beta_0x  < ex) * (ex < - beta_0x -  beta_1x) ).astype(np.float) 
print 'defiers: %s'%(d.sum()/N)

#dummy variable for always takers
a =   ( (- beta_0x < ex)  ).astype(np.float) 
print 'always takers: %s'%(a.sum()/N)

#dummy variable for never takers
n= ( (- beta_0x -  beta_1x > ex)  ).astype(np.float) 
print 'never takers: %s'%(n.sum()/N)

compliers: 0.625091
defiers: 0.0
always takers: 0.307898
never takers: 0.067011


### Part 11

In [9]:
x1c = x*c #x is 1 and a complier
x0c = (1-x)*c #x is 0 and a complier


y1c = (y*x1c).sum()/x1c.sum()
y0c =  (y*x0c).sum()/x0c.sum()

print 'LATE: %s'%(y1c - y0c)

LATE: 0.9971690555607438


### Part 12 

 The IV estimate is the LATE. The IV estimate is recovers the 'theoretical' average treatment effect. On the other hand, the OLS estimator is not equivalent to these.

### Part 13

Since $\beta_{1jy} =1$ for all $j$, the average treatment effect and the local average treatment effect should be the same. Everyone has the same treatment effect because there is no heterogeneity.

### Part 14

In [10]:
beta_1jy = np.random.normal(1, 1, size = N)

yj = beta_0y + beta_1jy*x + ey

### Part 15

$$E(y(1) -y(0)) = $$
$$E( (\beta_{0y} + \beta_{1jy} + \epsilon_{yj}) - (\beta_{0y} + \epsilon_{yj}) ) =$$
$$E(\beta_{1jy}) = 1$$


### Part 16

It does not. The individual coefficient is independent from selection process. However, selection is still endogenously determined

In [11]:
model = sm.OLS(yj, sm.add_constant(x))
model = model.fit()
print model.summary()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.172
Model:                            OLS   Adj. R-squared:                  0.172
Method:                 Least Squares   F-statistic:                 2.072e+05
Date:                Thu, 15 Nov 2018   Prob (F-statistic):               0.00
Time:                        09:41:53   Log-Likelihood:            -1.6189e+06
No. Observations:             1000000   AIC:                         3.238e+06
Df Residuals:                  999998   BIC:                         3.238e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9440      0.002    549.091      0.0

### Part 17

The IV estimator does recover the average treatment effect from question 15.

In [12]:
model =  IV2SLS(yj, sm.add_constant(x), sm.add_constant(z))
model = model.fit()
print model.summary()

                          IV2SLS Regression Results                           
Dep. Variable:                      y   R-squared:                       0.170
Model:                         IV2SLS   Adj. R-squared:                  0.170
Method:                     Two Stage   F-statistic:                 5.458e+04
                        Least Squares   Prob (F-statistic):               0.00
Date:                Thu, 15 Nov 2018                                         
Time:                        09:41:53                                         
No. Observations:             1000000                                         
Df Residuals:                  999998                                         
Df Model:                           1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0008      0.002    409.783      0.0

### Part 18


The selection process stays the same, so the groups should stay the same as well in parts 8 through 10.

The coefficient is random, but it does not depend on $\epsilon_x$ or $\epsilon_y$. Since selection into the treatment group is independent from the response to treatment, nothing changes.

### Part 19

In [13]:
yj1c = (yj*x1c).sum()/x1c.sum()
yj0c =  (yj*x0c).sum()/x0c.sum()

print 'LATE: %s'%(yj1c - yj0c)

LATE: 0.9971994618197411


### Part 20
15 19 and 17 are roughly the same. On the other hand, the OLS estimate is biased upward due to selection
 
### Part 21
The local average treatment effect is the average treatment effect. This is because the average treatment effect is the same among all groups. Heterogeniety is independent from the selection process.

### Part 22

In [14]:
beta_1iy = np.random.normal(1 + ex, 1, size = N)

yi = beta_0y + beta_1iy*x + ey

### Part 23
The average treatment effect increases. Because selection affects the treatment effect.

$$E(y(1) -y(0)) = $$

$$E( \beta_{1jy} )  = $$

$$E( E( \beta_{1jy} |\epsilon_{xj}) ) = $$

$$E( E(1 + \epsilon_{xj} |\epsilon_{xj}) ) =  1$$

### Part 24
Below we calculate the OLS estimator based on the numerical simulation. It does not recover the average treatment effect. We are biased upward

In [15]:
model = sm.OLS(yi, sm.add_constant(x))
model = model.fit()
print model.summary()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.265
Model:                            OLS   Adj. R-squared:                  0.265
Method:                 Least Squares   F-statistic:                 3.613e+05
Date:                Thu, 15 Nov 2018   Prob (F-statistic):               0.00
Time:                        09:41:55   Log-Likelihood:            -1.7578e+06
No. Observations:             1000000   AIC:                         3.516e+06
Df Residuals:                  999998   BIC:                         3.516e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9440      0.002    477.882      0.0

### Part 25

Because of the heterogeneity, we do not recover the average treatment effect.

In [16]:
model =  IV2SLS(yi, sm.add_constant(x), sm.add_constant(z))
model = model.fit()
print model.summary()

                          IV2SLS Regression Results                           
Dep. Variable:                      y   R-squared:                       0.163
Model:                         IV2SLS   Adj. R-squared:                  0.163
Method:                     Two Stage   F-statistic:                 1.501e+04
                        Least Squares   Prob (F-statistic):               0.00
Date:                Thu, 15 Nov 2018                                         
Time:                        09:41:56                                         
No. Observations:             1000000                                         
Df Residuals:                  999998                                         
Df Model:                           1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4622      0.003    488.782      0.0

### Part 26

The selection process stays the same, so the groups should stay the same as well in parts 8 through 10.

The average treatment effect increases. The IV estimate decreases. Neither actually recover the average treatment effect. The IV estimate is just the LATE.

### Part 27

The LATE is estimated by the IV as we can see below

In [17]:
yi1c = (yi*x1c).sum()/x1c.sum()
yi0c =  (yi*x0c).sum()/x0c.sum()

print 'LATE: %s'%(yi1c - yi0c)

LATE: 0.641608709852898


### Part 28

The IV estimate is the same as the LATE. It is different from the average treatment effect. Each of the groups has its own treatment effect.